READ CITY FROM CREATED DATABASE

In [12]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
load_dotenv() # Load .env into environment

schema = "city_workshop"
host = "127.0.0.1"
user = "root"
password = os.getenv("MYSQL_PASSWORD")
port = 3306
api_key = os.getenv("AEROBOX_API_KEY")


connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
# import your password-file if you have one
#from passwords import lianes_sql_key, aerodata_API_key

CREATE A FUCNCTION TO READ CITY

In [4]:
def get_cities(connection_string):
     return pd.read_sql("city", con=connection_string)


In [5]:
pd.read_sql("city", con=connection_string)

,City_id,city_name,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


READ THE CITY DF TO GET LATITUDE AND LONGITUDE FOR THE API AIRPORT CALL

In [13]:
cities_df = get_cities(connection_string)
cities_df    

,City_id,city_name,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


MAKE A FUNCTION CALL THE API

In [14]:

def icao_airport_codes(cities_df):
    icao_codes =  []
    city_id = []
    airport_name = []

    cities_df = pd.read_sql("city", con=connection_string)

    for index, row in cities_df.iterrows():
        latitude = row['Latitude'] # be aware of column names in your SQL table
        longitude = row['Longitude'] # be aware of column names in your SQL table
        
        
        
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location"

        querystring = {"lat": latitude,
                       "lon":longitude,
                       "radiusKm":"50",
                       "limit":"10",
                       "withFlightInfoOnly":"true"}

        headers = {
	        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
            "X-RapidAPI-Key": api_key #"YOUR_API_KEY_HERE"
        }

        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            airport_json = response.json()
        else:
            return response.reason
        
        # to append the data
        for data in airport_json['items']:
            icao_codes.append(data['icao'])
            city_id.append(row['City_id'])
            airport_name.append(data['name'])
            
        

    return pd.DataFrame({"city_id": city_id, "icao_codes": icao_codes, "airport_name" : airport_name })


In [15]:
cities_airport = icao_airport_codes(cities_df)
cities_airport

,city_id,icao_codes,airport_name
0,1,EDDT,Berlin -Tegel
1,1,EDDB,Berlin Brandenburg
2,2,EDDH,Hamburg
3,3,EDDM,Munich


send data to sql, ensure column name are correct

In [ ]:
#cities_airport.to_sql('cities_airport',
                 if_exists='append',
                 index=False,
                 con=connection_string)

In [16]:
airport_df= pd.read_sql("cities_airport", con=connection_string)
airport_df

,icao_codes,airport_name,city_id
0,EDDB,Berlin Brandenburg,1
1,EDDH,Hamburg,2
2,EDDM,Munich,3
3,EDDT,Berlin -Tegel,1
